In [1]:
import sys

In [2]:
sys.path.append('..')

In [3]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt
import arviz as az
import pymc3 as pm
from helpers import CausalModel, lfilter, create_standardizer_destandardizer
from copy import deepcopy
import seaborn as sns

In [4]:
%matplotlib Inline

In [5]:
%config InlineBackend.figure_format = 'svg'

In [6]:
np.set_printoptions(formatter={float: lambda x: '{:,.3f}'.format(x)})
pd.set_option('display.float_format', lambda x: '{:,.3f}'.format(x))

# The Haunted DAG & The Causal Terror

## Practice

### 6E1. 
List three mechanisms by which multiple regression can produce false inferences about causal effects.

1. Multicollinearity
2. Post-treatment Bias
3. Collider Bias

### 6E2. 
For one of the mechanisms in the previous problem, provide an example of your choice, perhaps from your own research.

Covid studies using only hospitalized patients data to causal inference

### 6E3.
List the four elemental confounds. Can you explain the conditional dependencies of each?

The fork, the collider, the pipe, and the descendant.

1. **The Fork** |  X <- Z -> Y

In a fork, Z is the common cause of X and Y. This common cause creates a correlation between X and Y. 
If we condition on Z (or in another words, put Z onto the regression), then learning X tell us nothing about Y. X and Y are independent, conditional on Z.

---

2. **The Pipe** | X -> Z -> Y

Conditional on Z, X and Y are independent. So to break the flow of information between X and Y you must add Z to your regression

---

3. **The Collider** | X -> Z <- Y

Unlike the previous ones, there is no association between X and Y unless you condition on Z. So to break the flow of information between X and Y you must remove Z from your regression.

---

4. **The Descendent** | 

Conditioning on D will also (partly) condition on Z, because D (as a child of Z) contains information about Z. Conditioning on D will also open the flow of information between X and Y because they are a collider on Z

### 6E4.
How is a biased sample like conditioning on a collider? Think of the example at the open of the chapter.

The DAG:

- T: Trustworthiness
- S: Selected
- N: Newsworthiness

T -> S <- N

Trustworthiness influences selection and Newsworthiness also influences selection. So when you condition on selection, it will appear as T influences N or N influences T. To break the flow of information between then you must remove S from your model. Or in the case of sampling, you should not only use the selected as a sample

### 6M1. 

Modify the DAG on page 186 to include the variable V, an unobserved cause of C and Y: C ← V → Y. Reanalyze the DAG. How many paths connect X to Y? Which must be closed? Which variables should you condition on now?